# JetBot Steering + Speed - JetBot Notebook

This notebook runs on your **JetBot** for data collection and inference.

## Cells
1. **Config** - All parameters (run first, always)
2. **Speed Tuning** - Find MAX_SPEED before blur (run once during setup)
3. **Data Collection** - Collect training data with joystick
4. **Inference** - Run trained model autonomously
5. **DAgger** - Collect corrections while model drives

In [ ]:
# =============================================================================
# CELL 1: CONFIGURATION (Run this first!)
# =============================================================================

import torch

# -----------------------------------------------------------------------------
# PATHS
# -----------------------------------------------------------------------------
DATASET_DIR = 'dataset_steering_speed_v1'
DAGGER_DIR = 'dataset_steering_speed_dagger'
MODEL_PATH = 'steering_speed_model_v1.pth'

# -----------------------------------------------------------------------------
# CAMERA
# -----------------------------------------------------------------------------
CAMERA_WIDTH = 640
CAMERA_HEIGHT = 480

# -----------------------------------------------------------------------------
# PREPROCESSING (same as steering-only project)
# -----------------------------------------------------------------------------
CROP_TOP = 0.20
CROP_BOTTOM = 0.00
CROP_LEFT = 0.08
CROP_RIGHT = 0.12
INPUT_SIZE = (224, 224)

# ImageNet normalization
IMAGENET_MEAN = [0.485, 0.456, 0.406]
IMAGENET_STD = [0.229, 0.224, 0.225]

# -----------------------------------------------------------------------------
# SPEED CONTROL
# -----------------------------------------------------------------------------
MIN_SPEED = 0.12    # Speed in corners (safe, tested)
MAX_SPEED = 0.25    # Speed on straights (tuned in Cell 2)

# Model outputs speed_factor (0 to 1), we scale it:
# actual_speed = MIN_SPEED + speed_factor * (MAX_SPEED - MIN_SPEED)

# -----------------------------------------------------------------------------
# STEERING CONTROL
# -----------------------------------------------------------------------------
STEERING_GAIN = 0.08

# -----------------------------------------------------------------------------
# TIMING
# -----------------------------------------------------------------------------
LOOP_HZ = 20
LOOP_SLEEP = 1.0 / LOOP_HZ

# -----------------------------------------------------------------------------
# JOYSTICK MAPPING (Logitech controller)
# -----------------------------------------------------------------------------
AXIS_STEERING = 0   # Left stick X
RT_BUTTON = 7       # Right trigger (analog 0-1)
RB_BUTTON = 5       # Right bumper
DEADZONE = 0.05

# -----------------------------------------------------------------------------
# DEVICE
# -----------------------------------------------------------------------------
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# -----------------------------------------------------------------------------
# HELPER FUNCTIONS
# -----------------------------------------------------------------------------
def speed_factor_to_actual(speed_factor):
    """Convert model output (0-1) to actual motor speed."""
    return MIN_SPEED + speed_factor * (MAX_SPEED - MIN_SPEED)

def actual_to_speed_factor(actual_speed):
    """Convert actual motor speed to model target (0-1)."""
    return (actual_speed - MIN_SPEED) / (MAX_SPEED - MIN_SPEED)

print("="*50)
print("STEERING + SPEED CONFIG")
print("="*50)
print(f"Device: {DEVICE}")
print(f"Speed range: {MIN_SPEED} - {MAX_SPEED}")
print(f"Dataset: {DATASET_DIR}")
print(f"Model: {MODEL_PATH}")
print("="*50)

In [ ]:
# =============================================================================
# CELL 2: SPEED TUNING (Run once during setup)
# =============================================================================
# Test different speeds on a straight section to find MAX_SPEED before blur.
#
# Instructions:
# 1. Place robot at start of a straight section
# 2. Press START
# 3. Gradually increase speed slider
# 4. Watch camera feed - when blur appears, press MARK BLUR POINT
# 5. Press STOP and update MAX_SPEED in Cell 1

import time
import threading
import cv2
import ipywidgets as widgets
from IPython.display import display
from jetbot import Camera, Robot, bgr8_to_jpeg

# Hardware
camera = Camera.instance(width=CAMERA_WIDTH, height=CAMERA_HEIGHT)
robot = Robot()

# Preprocessing
def preprocess(img):
    h, w = img.shape[:2]
    y0, y1 = int(h * CROP_TOP), int(h * (1 - CROP_BOTTOM))
    x0, x1 = int(w * CROP_LEFT), int(w * (1 - CROP_RIGHT))
    return cv2.resize(img[y0:y1, x0:x1], INPUT_SIZE)

def calculate_blur_metric(img):
    """Calculate Laplacian variance - lower = more blur."""
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return cv2.Laplacian(gray, cv2.CV_64F).var()

# Widgets
image_widget = widgets.Image(format='jpeg', width=224, height=224)
speed_slider = widgets.FloatSlider(
    value=MIN_SPEED, min=MIN_SPEED, max=0.35, step=0.01,
    description='Speed:', layout=widgets.Layout(width='400px')
)
info_label = widgets.Label(value=f'Speed: {MIN_SPEED:.2f} | Blur: --')
blur_indicator = widgets.HTML(value='<b style="color:green">Image Quality: Good</b>')
status_label = widgets.Label(value='Ready - Press START')

start_btn = widgets.Button(description='START', button_style='success')
stop_btn = widgets.Button(description='STOP', button_style='danger')
mark_btn = widgets.Button(description='MARK BLUR POINT', button_style='warning')

blur_point = {'speed': None}
running = False

def tuning_loop():
    global running
    blur_values = []
    
    while running:
        raw = camera.value
        processed = preprocess(raw)
        
        # Blur metric
        blur = calculate_blur_metric(processed)
        blur_values.append(blur)
        if len(blur_values) > 10:
            blur_values.pop(0)
        avg_blur = sum(blur_values) / len(blur_values)
        
        # Update blur indicator
        if avg_blur > 100:
            blur_indicator.value = f'<b style="color:green">Image Quality: Good ({avg_blur:.0f})</b>'
        elif avg_blur > 50:
            blur_indicator.value = f'<b style="color:orange">Image Quality: OK ({avg_blur:.0f})</b>'
        else:
            blur_indicator.value = f'<b style="color:red">Image Quality: BLURRY ({avg_blur:.0f})</b>'
        
        # Drive straight
        speed = speed_slider.value
        robot.left_motor.value = speed
        robot.right_motor.value = speed
        
        # Update UI
        image_widget.value = bgr8_to_jpeg(processed)
        info_label.value = f'Speed: {speed:.2f} | Blur: {avg_blur:.0f}'
        
        time.sleep(0.05)
    
    robot.stop()

def on_start(b):
    global running
    if not running:
        running = True
        status_label.value = 'RUNNING - Adjust speed slider'
        threading.Thread(target=tuning_loop, daemon=True).start()

def on_stop(b):
    global running
    running = False
    robot.stop()
    camera.stop()
    status_label.value = 'STOPPED'
    if blur_point['speed']:
        print(f"\n>>> Blur appeared at: {blur_point['speed']:.2f}")
        print(f">>> Recommended MAX_SPEED: {blur_point['speed'] - 0.02:.2f}")
    else:
        print(f"\n>>> No blur marked. Max tested: {speed_slider.value:.2f}")

def on_mark(b):
    blur_point['speed'] = speed_slider.value
    status_label.value = f'Blur marked at {blur_point["speed"]:.2f}'

start_btn.on_click(on_start)
stop_btn.on_click(on_stop)
mark_btn.on_click(on_mark)

print("Place robot on STRAIGHT section, press START, increase speed until blur.")
display(image_widget)
display(speed_slider)
display(info_label)
display(blur_indicator)
display(status_label)
display(widgets.HBox([start_btn, stop_btn, mark_btn]))

In [ ]:
# =============================================================================
# CELL 3: DATA COLLECTION
# =============================================================================
# Controls:
#   - RB (hold): Drive + Record
#   - Left Stick X: Steering
#   - RT: Speed (released=MIN, pressed=MAX)
#
# Filename format: {timestamp}_{steering:.3f}_{speed_factor:.3f}.jpg

import os
import time
import threading
import cv2
import numpy as np
import ipywidgets as widgets
from IPython.display import display
from jetbot import Camera, Robot, bgr8_to_jpeg

# Create directory
os.makedirs(DATASET_DIR, exist_ok=True)
existing = len([f for f in os.listdir(DATASET_DIR) if f.endswith('.jpg')])
print(f"Dataset: {DATASET_DIR} ({existing} existing images)")

# Hardware
camera = Camera.instance(width=CAMERA_WIDTH, height=CAMERA_HEIGHT)
robot = Robot()

# Preprocessing
def preprocess(img):
    h, w = img.shape[:2]
    y0, y1 = int(h * CROP_TOP), int(h * (1 - CROP_BOTTOM))
    x0, x1 = int(w * CROP_LEFT), int(w * (1 - CROP_RIGHT))
    return cv2.resize(img[y0:y1, x0:x1], INPUT_SIZE)

# Widgets
image_widget = widgets.Image(format='jpeg', width=224, height=224)
steering_slider = widgets.FloatSlider(
    value=0, min=-1, max=1, description='Steering:', disabled=True
)
speed_slider = widgets.FloatSlider(
    value=0, min=0, max=1, description='Speed:', disabled=True
)
actual_speed_label = widgets.Label(value=f'Actual: {MIN_SPEED:.2f}')
count_widget = widgets.IntText(value=existing, description='Images:', disabled=True)
status_label = widgets.Label(value='Ready - Press START')

start_btn = widgets.Button(description='START', button_style='success')
stop_btn = widgets.Button(description='STOP', button_style='danger')
controller = widgets.Controller()

running = False
image_count = existing

def collection_loop():
    global running, image_count
    
    while running:
        t0 = time.time()
        
        # Read joystick
        try:
            steering_raw = controller.axes[AXIS_STEERING].value
            rt_value = controller.buttons[RT_BUTTON].value
            rb_pressed = controller.buttons[RB_BUTTON].value > 0.5
        except:
            steering_raw, rt_value, rb_pressed = 0.0, 0.0, False
        
        # Process inputs
        steering = steering_raw if abs(steering_raw) > DEADZONE else 0.0
        speed_factor = max(0, min(1, rt_value))
        actual_speed = speed_factor_to_actual(speed_factor)
        
        # Get frame
        raw = camera.value
        processed = preprocess(raw)
        
        if rb_pressed:
            # Drive
            left = actual_speed + steering * STEERING_GAIN
            right = actual_speed - steering * STEERING_GAIN
            robot.left_motor.value = max(-1, min(1, left))
            robot.right_motor.value = max(-1, min(1, right))
            
            # Save
            filename = f"{int(time.time()*1000)}_{steering:.3f}_{speed_factor:.3f}.jpg"
            cv2.imwrite(os.path.join(DATASET_DIR, filename), processed)
            image_count += 1
            
            status_label.value = f'RECORDING ({image_count})'
        else:
            robot.stop()
            status_label.value = f'PAUSED - Hold RB ({image_count})'
        
        # Update UI
        image_widget.value = bgr8_to_jpeg(processed)
        steering_slider.value = steering
        speed_slider.value = speed_factor
        actual_speed_label.value = f'Actual: {actual_speed:.2f}'
        if image_count % 5 == 0:
            count_widget.value = image_count
        
        # Loop timing
        elapsed = time.time() - t0
        if LOOP_SLEEP - elapsed > 0:
            time.sleep(LOOP_SLEEP - elapsed)
    
    robot.stop()

def on_start(b):
    global running
    if not running:
        running = True
        threading.Thread(target=collection_loop, daemon=True).start()

def on_stop(b):
    global running
    running = False
    robot.stop()
    camera.stop()
    print(f"Stopped. {image_count} images in {DATASET_DIR}")

start_btn.on_click(on_start)
stop_btn.on_click(on_stop)

print("Controls: RB=Drive+Record, Left Stick=Steering, RT=Speed")
print(f"Speed range: {MIN_SPEED} (RT released) to {MAX_SPEED} (RT pressed)")
display(controller)
display(widgets.VBox([image_widget, steering_slider, speed_slider, actual_speed_label, count_widget, status_label]))
display(widgets.HBox([start_btn, stop_btn]))